# HDR (High Dynamic Range)
是因为我们的相机的动态范围有限，无法捕捉到太亮或者太暗的区域，所以我们需要多张图片来合成一张图片，这样就可以捕捉到更多的细节。

## 曝光序列生成和显示HDR图像 
Debevec Robertson 算法
## 曝光融合（Mertens）的方法
Mertens 生成低动态范围图像，并且不需要曝光时间数据

In [ ]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
# 将曝光图像加载到列表中
img_fn = ["../data/img0.jpg", "../data/img1.jpg", "../data/img2.jpg", "../data/img3.jpg"]
img_list = [cv.imread(fn) for fn in img_fn]

exposure_times = np.array([15.0, 2.5, 0.25, 0.0333], dtype=np.float32)

# 将曝光合成HDR图像
#  Debevec 算法
merge_debevec = cv.createMergeDebevec()
cal_debevec = cv.createCalibrateDebevec()
hdr_debevec = merge_debevec.process(img_list, times=exposure_times.copy())
crf_debevec = cal_debevec.process(img_list, times=exposure_times)

#  Robertson 算法
merge_robertson = cv.createMergeRobertson()
cal_robertson = cv.createCalibrateRobertson()
hdr_robertson = merge_robertson.process(img_list, times=exposure_times.copy())
crf_robertson = cal_robertson.process(img_list, times=exposure_times)

# 将HDR图像转换为8位图像
tonemap1 = cv.createTonemap(gamma=2.2)
res_debevec = tonemap1.process(hdr_debevec.copy())

tonemap1 = cv.createTonemap(gamma=2.2)
res_robertson = tonemap1.process(hdr_robertson.copy())


# 使用Mertens融合曝光
merge_mertens = cv.createMergeMertens()
res_mertens = merge_mertens.process(img_list)

# 转化数据类型为8-bit并保存 np.clip是为了防止数据溢出,
res_debevec_8bit = np.clip(res_debevec*255, 0, 255).astype('uint8')
res_robertson_8bit = np.clip(res_robertson*255, 0, 255).astype('uint8')
res_mertens_8bit = np.clip(res_mertens*255, 0, 255).astype('uint8')

img_total = np.concatenate((res_debevec_8bit, res_robertson_8bit, res_mertens_8bit), axis=1)

print(crf_debevec.shape)
print(crf_robertson.shape)
# 创建 x 轴的取值范围（0-255）
x = np.arange(256)

# 分离相机响应函数的通道
crf_debevec_r = crf_debevec[:, 0, 0]
crf_debevec_g = crf_debevec[:, 0, 1]
crf_debevec_b = crf_debevec[:, 0, 2]

crf_robertson_r = crf_robertson[:, 0, 0]
crf_robertson_g = crf_robertson[:, 0, 1]
crf_robertson_b = crf_robertson[:, 0, 2]

# 创建第一张图，显示 CRF Debevec 曲线
plt.figure(figsize=(10, 5))

# 绘制 CRF Debevec 曲线
plt.plot(x, crf_debevec_r, color='r', label='CRF Debevec (R)')
plt.plot(x, crf_debevec_g, color='g', label='CRF Debevec (G)')
plt.plot(x, crf_debevec_b, color='b', label='CRF Debevec (B)')

# 添加标题和图例
plt.title('CRF Debevec')
plt.legend()

# 创建第二张图，显示 CRF Robertson 曲线
plt.figure(figsize=(10, 5))

# 绘制 CRF Robertson 曲线
plt.plot(x, crf_robertson_r, color='r',  label='CRF Robertson (R)')
plt.plot(x, crf_robertson_g, color='g',  label='CRF Robertson (G)')
plt.plot(x, crf_robertson_b, color='b',  label='CRF Robertson (B)')

# 添加标题和图例
plt.title('CRF Robertson')
plt.legend()

# 显示图像
plt.show()


## 摄像机响应函数（CRF）

In [ ]:


# 将二维数组转换为一维数组
crf_debevec_1d = crf_debevec.flatten()
crf_robertson_1d = crf_robertson.flatten()

# 创建 x 轴的取值范围（0-255）
x = np.arange(256)

# 绘制 crf_debevec 曲线
plt.plot(x, crf_debevec_1d, color='r', label='CRF Debevec')

# 绘制 crf_robertson 曲线
plt.plot(x, crf_robertson_1d, color='b', label='CRF Robertson')

# 添加标题和图例
plt.title('Camera Response Function')
plt.legend()

# 显示图像
plt.show()


